# import module

In [34]:
import pandas as pd
import numpy as np
import os
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers.recurrent import LSTM
from keras.layers import Dropout
from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.utils import to_categorical

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [29]:
datas=pd.read_csv("../../etc/test/labeled_features.csv")

In [31]:
moving=datas[datas['label']=='moving']
observing=datas[datas['label']=='observing']
searching=datas[datas['label']=='searching']

In [33]:
moving['index']=0
observing['index']=1
searching['index']=2
LABELNUM=3

In [5]:
moving_train, moving_test=train_test_split(moving, test_size=0.3)
observing_train, observing_test=train_test_split(observing, test_size=0.3)
searching_train, searching_test=train_test_split(searching, test_size=0.3)

In [6]:
df_train=pd.concat([moving_train, observing_train, searching_train])
df_test=pd.concat([moving_test, observing_test, searching_test])

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   label         0 non-null      object 
 1   user          0 non-null      object 
 2   acc_range     0 non-null      float64
 3   acc_std       0 non-null      float64
 4   acc_skewness  0 non-null      float64
 5   acc_kurtosis  0 non-null      float64
 6   acc_energy    0 non-null      float64
 7   index         0 non-null      int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 0.0+ bytes


In [15]:
x_train=df_train.drop(['label', 'user', 'index'], axis=1)
y_train=df_train['index']
x_test=df_test.drop(['label', 'user', 'index'], axis=1)
y_test=df_test['index']

In [16]:
y_test_label=y_test

In [17]:
x_test = np.array(x_test)
#zeros  = np.zeros((x_train.shape[0] - x_test.shape[0], x_train.shape[1]))
#x_test = np.append(x_test, zeros, axis = 0)
x_test = pd.DataFrame(x_test)

In [18]:
x_train = np.reshape(x_train.values, [1, x_train.shape[0], x_train.shape[1]])
y_train = np.reshape(np.array(to_categorical(y_train)), [1, y_train.shape[0], LABELNUM])
x_test  = np.reshape(x_test.values, [1, x_test.shape[0], x_test.shape[1]])
y_test  = np.reshape(np.array(to_categorical(y_test)), [1, y_test.shape[0], LABELNUM])

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
BATCHSIZE = 32
EPOCHS    = 100
UNITNUM   = 50
optimizer = RMSprop(lr=0.01)
model = Sequential()
model.add(LSTM(UNITNUM, input_shape = (x_train.shape[1], x_train.shape[2]), return_sequences = True))
model.add(Dropout(0.2))
model.add(Dense(LABELNUM))
model.add(Activation("softmax"))
model.compile(loss = "mean_squared_error", optimizer = optimizer, metrics = ['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, batch_size = BATCHSIZE, epochs = EPOCHS)
modelName = model.__class__.__name__

In [ ]:
y_predicted = model.predict(x_test)
y_predicted = np.reshape(y_predicted, (x_train.shape[1], LABELNUM))
y_predicted = pd.DataFrame(y_predicted)
y_predicted = y_predicted.idxmax(axis = 1)
# y_predicted.to_csv("predicted_%s_LSTM.csv" % modelName)

In [ ]:
trueLabel      = y_test_label.values
predictedLabel = y_predicted.values
predictedLabel = predictedLabel[:y_test.shape[1]]

In [ ]:
trueLabel

In [ ]:
cm = confusion_matrix(trueLabel, predictedLabel)
sns.heatmap(cm, cbar = True, cmap = 'Greens')
plt.xlabel('predictedLabel')
plt.ylabel('trueLabel');
plt.savefig("CM_%s_LSTM.png" % modelName, format = 'png', dpi = 300)

In [ ]:
def plot_history_loss(fit):
    axL.plot(fit.history['loss'], label = "for training", color = 'dodgerblue')
    axL.set_title('model loss')
    axL.set_xlabel('epoch')
    axL.set_ylabel('loss')
    
def plot_history_accuracy(fit):
    axR.plot(fit.history['accuracy'], label = "for training", color = 'dodgerblue')
    axR.set_title('model accuracy')
    axR.set_xlabel('epoch')
    axR.set_ylabel('accuracy')

fig, (axL, axR) = plt.subplots(ncols = 2, figsize = (10,5))
plt.subplots_adjust(wspace = 0.5)
plot_history_loss(history)
plot_history_accuracy(history)
fig.savefig("loss_and_accuracy_LSTM.png", format = 'png', dpi = 300)